In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

tensorflow 는 2.3.1 버전으로 업그레이드 후 작성하여 주시기 바랍니다. \
tensorflow1을 이용하고 싶으신 분은 tensorflow.compat.v1을 import에서 사용하시면 됩니다.\
같이 첨부된 cheatsheet.ipynb에 이번과제 수행시 필요한 기능들에 대해서만 사용예시들을 적어놓았으므로 참고하시면 도움이 될 듯 합니다.

# Problem 1

TensorflowCNN 클래스의 함수들이 주어진 설명에 맞게 동작하도록 코드를 작성하십시오.

In [2]:
class TensorflowCNN(tf.Module):
    def __init__(self, filter_list, learning_rate):
        '''
        filter_list를 저장해 놓고 __call__ 호출시 사용합니다.
        filter_list의 마지막 텐서의 out channel수가 class의 개수가 됩니다. 
        learning_rate는 fit에서 training시에 사용합니다.
        
        Inputs: 
        - filter_list: [filter_height, filter_weight, in_channels, out_channels] 의 shape을 가진 텐서들의 리스트
        - learning_rate: float 값으로 optimizer의 learning rate로 설정해 줍니다.
        '''
        self.filter_list = filter_list
        self.lr = learning_rate
        return
    
    def __call__(self, images):
        '''
        init에서 저장해 놓은 tensor들을 filter로 하여 convolution operation을 수행합니다.
        마지막에 1, 2 axis방향으로 reduce_mean을 수행하여 [batch_size, optdim]의 shape을 가지도록 하고 1 axis로 softmax를 수행합니다.
        convolution이외의 opearation은 사용하지 않습니다.
        
        Inputs: 
        - images: [batch_size, height, width, channel]의 shape을 가진 tensor입니다.
        
        Returns: 
        - result: convolution operation을 수행한 결과로 [batch_size, optdim]의 shape을 가진 tensor입니다.
        '''
        #### Method 1 : Just using operations ####
        temp = tf.constant(images, dtype=tf.float32)
        for i in range(np.shape(self.filter_list)[0]):
            temp = tf.nn.conv2d(temp, self.filter_list[i], strides=[1,1,1,1], padding = "VALID")
            temp = tf.nn.relu(temp)
        temp = tf.math.reduce_mean(temp,axis=1)
        temp = tf.math.reduce_mean(temp,axis=1)
        result = tf.nn.softmax(temp,axis=1)
        
        return result
    
    def fit(self, images, labels, epochs):
        '''
        images, labels을 이용하여 epochs 번 업데이트를 수행합니다.
        loss는 cross entropy를 이용하여 optimizer는 SGD를 이용합니다.
        learning_rate는 init에서 저장한 값을 이용합니다.
        
        Inputs: 
        - images: [batch_size, height, width, channel]의 shape을 가진 tensor입니다.
        - labels: [batch_size]의 shape을 가진 integer-valed tensor입니다. 0이상 클래스 수 미만의 값을 가집니다.
        - epochs: integer값으로 update step수를 나타냅니다.
        
        Returns: 
        - losses: float의 리스트로 
        '''
        
        losses = np.zeros([epochs])
        optimizer = tf.keras.optimizers.SGD(learning_rate = self.lr)
        
        for i in range(epochs):
            with tf.GradientTape() as tape:
                loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(labels, self(images)))
            gradients = tape.gradient(loss, self.filter_list)
            optimizer.apply_gradients(zip(gradients,self.filter_list))
            losses[i]=loss.numpy()
        
        return losses
    
    def save(self):
        '''
        saved_model이 아닌 checkpoint를 이용하여 저장합니다.
        '''
        self.ckpt_map1 = {'var' : self.filter_list, 'model' : self}
        self.ckpt1 = tf.train.Checkpoint(**self.ckpt_map1)
        self.ckpt_manager = tf.train.CheckpointManager(checkpoint=self.ckpt1,directory='ckpt',max_to_keep=1)
        self.ckpt_manager.save()
        return
    
    def restore(self):
        '''
        마지막 save함수가 호출 되었을 때의 값으로 variable의 값들을 set해 줍니다.
        '''
        self.ckpt1.restore(self.ckpt_manager.latest_checkpoint)
        return

# Problem 2

cifar100 dataset에 대해서 tensorflow를 이용하여 모델을 작성하고 training하여서 tf.saved_model.save(model, 'model')을 통해 저장하십시오.
- training이 완료된 model은 프로젝트 폴더 하위에 위치한 'model' 디렉토리에 저장되어 있어야 합니다.
- loaded_model = tf.saved_model.load('model')의 방법을 통해서 불러 올 수 있어야 하고 불러온 모델의 accuracy가 0.85 이상이면 정답으로 처리하겠습니다.
- **model를 training을 완료하고 저장후에는 "반드시 주석처리" 하여서 notebook import시에 실행이 되지 않도록 해 주세요.**

In [8]:
# train= tfds.load(name="cifar100", split="train", shuffle_files=True)#, as_supervised=True)

# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(32,kernel_size=[3, 3],activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
#     tf.keras.layers.Conv2D(64,kernel_size=[3, 3],activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
#     tf.keras.layers.Conv2D(128,kernel_size=[3, 3],activation='relu'),
#     tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(256,activation='relu'),
#     tf.keras.layers.Dense(128,activation='relu'),
#     tf.keras.layers.Dense(100,activation='softmax')
# ])

# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
# model.compile(optimizer=optimizer, loss=loss_fn)



# for epoch in range(30):
#     for idx, batch in enumerate(train.batch(100)):
#         image, label = batch["image"], batch["label"]
#         image = tf.cast(image, tf.float32)
#         model.fit(image,label, epochs=1, verbose=0)
#         if(idx%100==0): model.fit(image,label, epochs=1, verbose=1)
#     print("epoch : %d"%(epoch))

# tf.saved_model.save(model,"model")

Learning Start
4/4 [==============================] - 0s 7ms/step - loss: 3.7134
epoch : 0
4/4 [==============================] - 0s 9ms/step - loss: 3.4603
epoch : 1
4/4 [==============================] - 0s 7ms/step - loss: 3.4541
epoch : 2
4/4 [==============================] - 0s 8ms/step - loss: 3.2881
epoch : 3
4/4 [==============================] - 0s 8ms/step - loss: 3.1177
epoch : 4
4/4 [==============================] - 0s 11ms/step - loss: 3.0198
epoch : 5
4/4 [==============================] - 0s 7ms/step - loss: 2.8431
epoch : 6
4/4 [==============================] - 0s 7ms/step - loss: 2.7257
epoch : 7
4/4 [==============================] - 0s 7ms/step - loss: 2.8439
epoch : 8
4/4 [==============================] - 0s 7ms/step - loss: 2.8524
epoch : 9
4/4 [==============================] - 0s 8ms/step - loss: 2.7222
epoch : 10
4/4 [==============================] - 0s 10ms/step - loss: 2.6288
epoch : 11
4/4 [==============================] - 0s 8ms/step - loss: 2.5350
epo

4/4 [==============================] - 0s 7ms/step - loss: 1.9997
epoch : 24
4/4 [==============================] - 0s 8ms/step - loss: 2.1358
epoch : 25
4/4 [==============================] - 0s 7ms/step - loss: 1.8489
epoch : 26
4/4 [==============================] - 0s 7ms/step - loss: 1.9108
epoch : 27
4/4 [==============================] - 0s 8ms/step - loss: 1.9058
epoch : 28
4/4 [==============================] - 0s 7ms/step - loss: 1.9472
epoch : 29
INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


# Submission

프로젝트 폴더를 HW3_20XX_XXXXX.zip으로 HW3_본인학번.zip 압축하여서 etl을 통해 제출하시면 됩니다.\
**notebook을 import하여 정상동작여부를 확인하므로 import문을 제외하고는 notebook import시에 실행되는 코드가 없도록 확인해 주세요**